In [0]:
# set up environment 

In [ ]:
# Install necessary packages
!pip install sagemaker torch torchvision

# Import SageMaker and other required libraries
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

# Get the SageMaker execution role
role = get_execution_role()

In [ ]:
# specify S3 inputs 

In [ ]:
s3_bucket = 'dis-sagemaker'
s3_prefix = 'dis'

In [ ]:
# build and push container to ECR

In [ ]:
image_name = 'dis-image-segmentation'

!sh ./src/build_and_push.sh $image_name

with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

In [ ]:
# Set up the PyTorch estimator

In [ ]:
estimator = PyTorch(entry_point='train.py',
                    role=role,
                    framework_version='1.8.1',
                    train_instance_count=1,
                    train_instance_type='ml.p3.2xlarge',  # Choose an appropriate instance type
                    output_path=f's3://{s3_bucket}/{s3_prefix}/output',
                    hyperparameters={
                        'epochs': 5
                    },
                    image_uri=container)

# Start the training job
estimator.fit({'training': f's3://{s3_bucket}/{s3_prefix}/training-data'})